# DX 704 Week 1 Project

This week's project will build a portfolio risk and return model, and make investing recommendations for hypothetical clients.
You will collect historical data, estimate returns and risks, construct efficient frontier portfolios, and sanity check the certainty of the maximum return portfolio.

The full project description and a template notebook are available on GitHub at the following link.

https://github.com/bu-cds-dx704/dx704-project-01


Feel free to use optimization tools or libraries (such as CVXOPT or scipy.optimize) to perform any calculations required for this mini project.

### Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Data

Collect historical monthly price data for the last 24 months covering 6 different stocks.
The data should cover 24 consecutive months including the last month that ended before this week's material was released on Blackboard.
To be clear, if a month ends between the Blackboard release and submitting your project, you do not need to add that month.

The six different stocks must include AAPL, SPY and TSLA.
At least one of the remaining 3 tickers must start with the same letter as your last name (e.g. professor Considine could use COIN).
This is to encourage diversity in what stocks you analyze; if you discuss this project with classmates, please make sure that you pick different tickers to differentiate your work.
Do not pick stocks with fewer than 24 consecutive months of price data.

In [1]:
# YOUR CHANGES HERE

import cvxpy as cp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf

tickers = ["AAPL", "SPY", "TSLA", "FNMA", "FIGR.PVT", "F"]

historical_prices = None

for t in tickers:
    ticker = yf.Ticker(t)
    history =  ticker.history(period="2y")

    monthly_prices = history['Close'].resample('ME').last()

    if historical_prices is None:
        historical_prices = monthly_prices
    else:
        historical_prices = pd.concat([historical_prices, monthly_prices], axis=1)

historical_prices.columns = tickers
historical_prices.index = pd.to_datetime(historical_prices.index).strftime("%Y-%m-%d")
historical_prices = historical_prices.iloc[1:]

historical_prices

,AAPL,SPY,TSLA,FNMA,FIGR.PVT,F
Date,,,,,,
2023-10-31,169.113525,408.794373,200.839996,0.75,3.000000,8.581726
2023-11-30,188.355316,446.135223,240.080002,0.74,3.000000,9.030617
2023-12-31,190.913681,466.503662,248.479996,1.07,3.000000,10.729358
2024-01-31,182.851883,473.933411,187.289993,1.26,3.000000,10.315675
2024-02-29,179.461273,498.666504,201.880005,1.27,13.220000,11.244849
2024-03-31,170.257355,514.973877,175.789993,1.57,13.220000,12.004148
2024-04-30,169.115555,494.210358,183.279999,1.47,13.220000,10.982709
2024-05-31,191.138168,519.207336,178.080002,1.55,13.220000,11.097805
2024-06-30,209.401917,537.525024,197.880005,1.43,13.220000,11.472918


Save the data as a TSV file named "historical_prices.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
The date should be the last trading day of the month, so it may not be the last day of the month.
For example, the last trading day of November 2024 was 2024-11-29.
The remaining columns should contain the adjusted closing prices of the corresponding stock tickers on that day.


In [2]:
# YOUR CHANGES HERE

# Save historical_prices as a TSV file
historical_prices_tsv = historical_prices.reset_index().rename(columns={'Date': 'date'})
historical_prices_tsv.to_csv('historical_prices.tsv', sep="\t", index=False)

Submit "historical_prices.tsv" in Gradescope.

## Part 2: Calculate Historical Asset Returns

Calculate the historical asset returns based on the price data that you previously collected.

In [3]:
# YOUR CHANGES HERE

historical_returns = historical_prices.pct_change().dropna().reset_index().rename(columns={'Date': 'date'})
historical_returns

,date,AAPL,SPY,TSLA,FNMA,FIGR.PVT,F
0,2023-11-30,0.113780,0.091344,0.195379,-0.013333,0.000000,0.052308
1,2023-12-31,0.013583,0.045655,0.034988,0.445946,0.000000,0.188109
2,2024-01-31,-0.042227,0.015926,-0.246257,0.177570,0.000000,-0.038556
3,2024-02-29,-0.018543,0.052187,0.077901,0.007937,3.406667,0.090074
4,2024-03-31,-0.051286,0.032702,-0.129235,0.236221,0.000000,0.067524
5,2024-04-30,-0.006706,-0.040320,0.042608,-0.063694,0.000000,-0.085090
6,2024-05-31,0.130222,0.050580,-0.028372,0.054422,0.000000,0.010480
7,2024-06-30,0.095553,0.035280,0.111186,-0.077419,0.000000,0.033801
8,2024-07-31,0.054411,0.012109,0.172781,-0.104895,0.002269,-0.137161
9,2024-08-31,0.032353,0.023366,-0.077390,-0.062500,0.000000,0.050305


Save the data as a TSV file named "historical_returns.tsv" and include a header row with the column names "date" and the 6 stock ticker symbols.
Each row should have the date at the end of the month and the corresponding *relative* price changes.
For example, if the previous price was \$100 and the new price is \$110, the return value should be 0.10.
There should only be 23 rows of data in this file, since they are computed as the differences of 24 prices.

In [4]:
# YOUR CHANGES HERE
# historical_returns_tsv = historical_returns.reset_index().rename(columns={'Date': 'date'})
historical_returns.to_csv('historical_returns.tsv', sep="\t", index=False)

Submit "historical_returns.tsv" in Gradescope.

## Part 3: Estimate Returns

Estimate the expected returns for each asset using the previously calculated return data.
Just compute the average (mean) return for each asset over your data set; do not use other estimators that have been mentioned.
This will serve as your estimate of expected return for each asset.

In [5]:
historical_returns.columns

Index(['date', 'AAPL', 'SPY', 'TSLA', 'FNMA', 'FIGR.PVT', 'F'], dtype='object')

In [6]:
# YOUR CHANGES HERE

# estimated_returns = pd.DataFrame({'asset': historical_returns.columns, 'estimated_return': historical_returns.mean()})
estimated_returns = historical_returns.drop('date', axis=1).mean().reset_index().rename(columns={'index': 'asset', 0: 'estimated_return'})
# estimated_returns = estimated_returns.reset_index(drop=True)
estimated_returns

,asset,estimated_return
0,AAPL,0.016968
1,SPY,0.020758
2,TSLA,0.036103
3,FNMA,0.166345
4,FIGR.PVT,0.596435
5,F,0.016225


Save the estimated returns in a TSV file named "estimated_returns.tsv" and include a header row with the column names "asset" and "estimated_return".

In [7]:
# YOUR CHANGES HERE

estimated_returns.to_csv("estimated_returns.tsv", sep="\t", index=False)

Submit "estimated_returns.tsv" in Gradescope.

## Part 4: Estimate Risk

Estimate the covariance matrix for the asset returns to understand how the assets move together.

In [8]:
# YOUR CHANGES HERE

estimated_covariance = historical_returns.drop('date', axis=1).cov()
estimated_covariance

,AAPL,SPY,TSLA,FNMA,FIGR.PVT,F
AAPL,0.003623,0.000784,0.003075,-0.002122,-0.001532,-0.000060
SPY,0.000784,0.001225,0.001971,0.003686,-0.010873,0.001348
TSLA,0.003075,0.001971,0.024358,0.018662,-0.135266,0.000173
FNMA,-0.002122,0.003686,0.018662,0.110091,-0.019752,0.010178
FIGR.PVT,-0.001532,-0.010873,-0.135266,-0.019752,4.817232,-0.006611
F,-0.000060,0.001348,0.000173,0.010178,-0.006611,0.005268


Save the estimated covariances to a TSV file named "estimated_covariance.tsv".
The header row should have a blank column name followed by the names of the assets.
Each data row should start with the name of an asset for that row, and be followed by the individual covariances corresponding to that row and column's assets.
(This is the format of pandas's `to_csv` method with `sep="\t"` when used on a covariance matrix as computed in the examples.)

In [9]:
# YOUR CHANGES HERE

estimated_covariance.to_csv("estimated_covariance.tsv", sep="\t")

Submit "estimated_covariance.tsv" in Gradescope.

## Part 5: Construct the Maximum Return Portfolio

Compute the maximum return portfolio based on your previously estimated risks and returns.

In [10]:
# YOUR CHANGES HERE

# Expected returns vector:
mu_max = estimated_returns['estimated_return'].to_numpy()
assets = estimated_returns['asset'].to_numpy()

n_max = len(mu_max)
x_max = cp.Variable(n_max)

# Maximum return
max_objective = cp.Maximize(mu_max @ x_max)
max_constraints = [x_max >= 0, cp.sum(x_max) == 1]
max_prob = cp.Problem(max_objective, max_constraints)
max_prob.solve()

# Segment into two columns
maximum_return = pd.DataFrame({'asset': estimated_returns['asset'].values, 'allocation': x_max.value})
maximum_return

,asset,allocation
0,AAPL,-4.006404e-11
1,SPY,-4.104668e-11
2,TSLA,-4.530386e-11
3,FNMA,-1.096514e-10
4,FIGR.PVT,1.000000e+00
5,F,-3.987460e-11


Save the maximum return portfolio in a TSV file named "maximum_return.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [11]:
# YOUR CHANGES HERE

maximum_return.to_csv("maximum_return.tsv", sep="\t", index=False)

Submit "maximum_return.tsv" in Gradescope.

## Part 6: Construct the Minimum Risk Portfolio

Compute the minimum return portfolio based on your previously estimated risks and returns.

In [12]:
# YOUR CHANGES HERE

# Expected returns vector:
mu_min = estimated_returns['estimated_return'].to_numpy()
assets = estimated_returns['asset'].to_numpy()

n_min = len(mu_min)
x_min = cp.Variable(n_min)

# Minimum return
min_objective = cp.Minimize(x_min.T @ estimated_covariance.to_numpy() @ x_min)
min_constraints = [x_min >= 0, cp.sum(x_min) == 1]
min_prob = cp.Problem(min_objective, min_constraints)
min_prob.solve()

# Segment into two columns
minimum_return = pd.DataFrame({'asset': estimated_returns['asset'].values, 'allocation': x_min.value})
minimum_return

,asset,allocation
0,AAPL,1.278606e-01
1,SPY,8.698983e-01
2,TSLA,2.670663e-21
3,FNMA,-1.484087e-19
4,FIGR.PVT,2.241090e-03
5,F,-5.536586e-20


Save the minimum risk portfolio in a TSV file named "minimum_risk.tsv".
The header row should have two columns, "asset" and "allocation".
The allocation values should sum up to one.


In [13]:
# YOUR CHANGES HERE

minimum_return.to_csv("minimum_risk.tsv", sep="\t", index=False)

Submit "minimum_risk.tsv" in Gradescope.

## Part 7: Build Efficient Frontier Portfolios

Compute 101 portfolios along the mean-variance efficient frontier with evenly spaced estimated returns.
The first portfolio should be the minimum risk portfolio from part 4, and the last portfolio should be the maximum return portfolio from part 3.
The estimated return of each portfolio should be higher than the previous by one percent of the difference between the first and last portfolios.
That is, the estimated return of the portfolios should be similar to `np.linspace(min_risk_return, max_return, 101)`.


In [14]:
estimated_returns.columns

Index(['asset', 'estimated_return'], dtype='object')

In [15]:
# YOUR CHANGES HERE

# Convert the assets to a list
ef_assets = estimated_returns['asset'].tolist()

# Returns vector
returns = (estimated_returns
           .set_index("asset")
           .loc[ef_assets, "estimated_return"]
           .astype(float)
           .to_numpy())

# Align covariance to the proper order
sigma = estimated_covariance.loc[ef_assets, ef_assets].to_numpy()
n = len(ef_assets)

# Scalars for min and max returns
estimated_return_minimum_risk = float(np.asarray(x_min.value, dtype=float).ravel() @ returns)
estimated_return_maximum_risk = float(returns.max())

# efficient frontier
ef_variances, ef_portfolios = [], []

for r in np.linspace(estimated_return_minimum_risk, estimated_return_maximum_risk, 101):
    # create variables to optimize
    x_r = cp.Variable(n)

    # formulate linear program
    prob_r = cp.Problem(cp.Minimize(x_r.T @ sigma @ x_r),
                        [0 <= x_r,
                         cp.sum(x_r) == 1,
                         x_r.T @ returns == r])
    variance_opt = prob_r.solve()
    w = np.asarray(x_r.value, dtype=float).ravel()
    w = np.clip(w, 0, None)
    if w.sum() > 0:
        w = w / w.sum()
    ef_variances.append(variance_opt)
    ef_portfolios.append(w)

# Matrix of weights (101 x n)
weights = np.vstack(ef_portfolios)

# Force the last frontier point to be the exact maximum-return portfolio
imax = int(np.argmax(returns))     # index of asset with highest mean return
one_hot = np.zeros(n)
one_hot[imax] = 1.0
weights[-1, :] = one_hot

# Portfolio returns (use solved weights, not target r)
ef_returns = weights @ returns

# Portfolio standard deviations
ef_vars = np.array([w @ sigma @ w for w in weights], dtype=float)
ef_vars = np.clip(ef_vars, 0, None)
ef_stds = np.sqrt(ef_vars)

# Setup the efficient frontier
efficient_frontier = pd.DataFrame(weights, columns=ef_assets)
efficient_frontier.insert(0, "risk", ef_stds)
efficient_frontier.insert(0, "return", ef_returns)
efficient_frontier.insert(0, "index", np.arange(weights.shape[0], dtype=int))

# Reorganize the efficient frontier dataframe
efficient_frontier = efficient_frontier[['index', 'return', 'risk'] + ef_assets]
efficient_frontier

,index,return,risk,AAPL,SPY,TSLA,FNMA,FIGR.PVT,F
0,0,0.021563,0.033790,0.127861,0.869898,0.000000e+00,0.000000,0.002241,0.0
1,1,0.027312,0.036837,0.123708,0.845343,4.067508e-20,0.025096,0.005853,0.0
2,2,0.033061,0.042638,0.137681,0.795056,4.627318e-03,0.054185,0.008451,0.0
3,3,0.038810,0.050131,0.145491,0.745237,1.635055e-02,0.081705,0.011216,0.0
4,4,0.044558,0.058664,0.153302,0.695417,2.807378e-02,0.109225,0.013982,0.0
...,...,...,...,...,...,...,...,...,...
96,96,0.573440,2.077068,0.000000,0.000000,0.000000e+00,0.053465,0.946535,0.0
97,97,0.579189,2.106491,0.000000,0.000000,0.000000e+00,0.040099,0.959901,0.0
98,98,0.584938,2.135924,0.000000,0.000000,0.000000e+00,0.026733,0.973267,0.0
99,99,0.590686,2.165367,0.000000,0.000000,0.000000e+00,0.013366,0.986634,0.0


Save the portfolios in a TSV file named "efficient_frontier.tsv".
The header row should have columns "index", "return", "risk", and all the asset tickers.
Each data row should have the portfolio index (0-100), the estimated return of the portfolio, the estimated standard deviation (not variance) of the portfolio, and all the asset allocations (which should sum to one).

In [16]:
# YOUR CHANGES HERE

efficient_frontier.to_csv("efficient_frontier.tsv", sep="\t", index=False)

In [17]:
efficient_frontier.shape

(101, 9)

Submit "efficient_frontier.tsv" in Gradescope.

## Part 8: Check Maximum Return Portfolio Stability

Check the stability of the maximum return portfolio by resampling the estimated risk/return model.

Repeat 1000 times -
1. Use `np.random.multivariate_normal` to generate 23 return samples using your previously estimated risks and returns.
2. Estimate the return of each asset using that resampled return history.
3. Check which asset had the highest return in those resampled estimates.

This procedure is a reduced and simplified version of the Michaud resampled efficient frontier procedure that takes uncertainty in the risk model into account.

In [18]:
# # YOUR CHANGES HERE

# np.random.seed(704_1) 

# mu = estimated_returns['estimated_return'].to_numpy()

# # Simulation setup
# n_sims = 1000
# n_samples = 23

# # Random Generator
# range = np.random.default_rng(7041)

# # Draw the returns
# draws = range.multivariate_normal(mean=mu, cov=sigma, size=(n_sims, n_samples))

# # Get the means for each draw per-asset
# sample_means = draws.mean(axis=1)

# # Get the winning asset from each simulation
# winners_index = sample_means.argmax(axis=1)

# # Count the wins and compute the stability of the theoretical portfolio tailored for maximum return
# index_max = int(mu.argmax())
# wins = np.bincount(winners_index, minlength=len(assets))
# stability = wins[index_max] / n_sims

# # Table of maximum return predictions
# winner_table = (
#     pd.DataFrame({'asset': assets, 'wins': wins})
#         .assign(probability=lambda d: d['wins'] / n_sims)
#         .drop('wins', axis=1)
#         .reset_index(drop=True)
# )

# winner_table

In [19]:
# Create variables to optimize
mu = estimated_returns["estimated_return"].astype(float).to_numpy()
Sigma = np.asarray(estimated_covariance, dtype=float)
print(Sigma.shape)
n = Sigma.shape[0]
T = 23

max_ef_variances = []
max_ef_returns = []
max_ef_portfolios = []

for r in np.linspace(estimated_return_minimum_risk, estimated_return_maximum_risk, 1000):
    x_r = cp.Variable(n)

    # formulate linear program
    prob_r = cp.Problem(cp.Minimize(x_r.T @ Sigma @ x_r),
                        [0 <= x_r,
                         cp.sum(x_r) == 1,
                         mu @ x_r == r])
    max_ef_variances.append(prob_r.solve())
    max_ef_returns.append(r)
    max_ef_portfolios.append(x_r.value)

max_ef_portfolios = np.asarray(max_ef_portfolios)

# print(len(max_ef_variances), len(max_ef_returns), len(max_ef_portfolios))

# Get the asset names
asset_names = list(estimated_covariance.columns)

# Take the highest return portfolio
max_portfolio = max_ef_portfolios[-1]

# Build the dataframe
max_return_probabilities = pd.DataFrame({
    "asset": asset_names, 
    "probability": max_portfolio
})
max_return_probabilities

(6, 6)


,asset,probability
0,AAPL,-1.909014e-07
1,SPY,1.622442e-08
2,TSLA,3.219923e-07
3,FNMA,-2.500652e-08
4,FIGR.PVT,1.000000e+00
5,F,-1.138403e-07


Save a file "max_return_probabilities.tsv" with the distribution of highest return assets.
The header row should have columns "asset" and "probability".
There should be a data row for each asset and its sample probability of having the highest return based on those 1000 resampled estimates.


In [20]:
# YOUR CHANGES HERE

# winner_table.to_csv('max_return_probabilities.tsv', sep='\t', index=False)
max_return_probabilities.to_csv('max_return_probabilities.tsv', sep='\t', index=False)

Submit "max_return_probabilities.tsv" in Gradescope.

## Part 9: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgements.txt" in Gradescope.

## Part 10: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

Submit "project.ipynb" in Gradescope.